In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
# NLTK VADER for sentiment analysis
import nltk
nltk.downloader.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import numpy as np
import tensorflow as tf

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [ ]:
# !unzip /content/archive.zip

Archive:  /content/archive.zip
replace News_sentiment_Jan2017_to_Apr2021.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
# news=pd.read_csv("/content/News_sentiment_Jan2017_to_Apr2021.csv")
news=pd.read_csv("/content/ICICI_BANK_NEWS_HEADLINES.csv", encoding='ISO-8859-1')

In [ ]:
news=pd.DataFrame(news)

In [ ]:
news.head()

,Date,Title
0,2023-02-05,Trade setup for Wednesday : Top 15 things to k...
1,2023-02-05,Amazon Sale : Amazon Kickstarter Deals : Shop ...
2,2023-02-05,"Stock market today : Sensex , Nifty extend win..."
3,2023-02-05,stocks to buy now : Hemang Jani top 6 bets in ...
4,2023-02-05,active stocks : Share market update : Most act...


In [ ]:
news['Date'] = pd.to_datetime(news['Date']).dt.date

In [ ]:
import yfinance as yf
import datetime

# Define the stock symbol (ICICI Bank's symbol is "ICICIBANK.NS" for NSE or "ICICIBANK.BO" for BSE)
stock_symbol = "ICICIBANK.NS"

# Define the start and end dates
start_date = datetime.datetime(2017, 1, 1)
end_date = datetime.datetime(2023, 12, 31)

# Fetch historical data
df = yf.download(stock_symbol, start=start_date, end=end_date)

# Extract the closing prices







[*********************100%%**********************]  1 of 1 completed


In [ ]:
df['Indicator'] = df['Open'] - df['Close']
df['Indicator'] = df['Indicator'].apply(lambda x: 1 if x > 0 else -1)

In [ ]:
print(df)

            Indicator
2017-01-02          1
2017-01-03         -1
2017-01-04          1
2017-01-05         -1
2017-01-06          1
...               ...
2023-08-29          1
2023-08-30          1
2023-08-31          1
2023-09-01         -1
2023-09-04          1

[1651 rows x 1 columns]


In [ ]:
df=df.drop(['Open','High','Low','Close','Adj Close','Volume'],axis=1)

In [ ]:
df.index = pd.to_datetime(df.index)

# Convert the datetime index to date objects
df.index = df.index.date

In [ ]:
news=news.set_index('Date')

In [ ]:
merged_df = pd.merge(df, news, left_index=True, right_index=True, how='inner')


In [ ]:
print(news)

                                                        Title
Date                                                         
2023-02-05  Trade setup for Wednesday : Top 15 things to k...
2023-02-05  Amazon Sale : Amazon Kickstarter Deals : Shop ...
2023-02-05  Stock market today : Sensex , Nifty extend win...
2023-02-05  stocks to buy now : Hemang Jani top 6 bets in ...
2023-02-05  active stocks : Share market update : Most act...
...                                                       ...
2017-10-01  5 cues from F & O market : Fresh Put writing a...
2017-10-01  Nifty above 8250 , Sensex climbs over 100 pts ...
2017-10-01  Sensex rallies over 100 points ; Nifty50 above...
2017-10-01  Eleven stocks in focus on Tuesday , 10 January...
2017-09-01  2017 promises to attract discerning investor t...

[16709 rows x 1 columns]


In [ ]:
print(merged_df)

            Indicator                                              Title
2017-01-02          1  Budget 2017 : Balancing growth with fiscal pru...
2017-01-02          1                                  Union Budget 2017
2017-01-02          1  Technical Analysis Premium Mailer : Premium St...
2017-01-02          1     ICICI Bank resumes UPI transactions on PhonePe
2017-01-02          1                                        Mcap of BSE
...               ...                                                ...
2023-09-01         -1  Nifty : Nifty may trade in 17 , 777 - 18 , 250...
2023-09-01         -1  Best tax saving options : Comparison of 10 inv...
2023-09-04          1  Trade setup for Monday : Top 15 things to know...
2023-09-04          1  Lenders likely to post robust Q4 numbers ; PSU...
2023-09-04          1  Mcap of eight of top 10 valued firms climbs Rs...

[13641 rows x 2 columns]


In [ ]:
news=news.drop(['URL','confidence','Unnamed: 5'],axis=1)

In [ ]:
replace_dict = {'POSITIVE': 1, 'NEGATIVE': -1}

In [ ]:
news['sentiment'] = news['sentiment'].replace(replace_dict)

In [ ]:
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

In [ ]:
merged_df['tokens'] = merged_df['Title'].apply(word_tokenize)

In [ ]:
model1 = Word2Vec(sentences=merged_df['tokens'], vector_size=200, window=5, min_count=1, workers=4)

In [ ]:
news_train = news[news['Date'].dt.year < 2020]
news_test = news[news['Date'].dt.year > 2020]

In [ ]:
target_shape = (20, 200)  # Adjust dimensions as needed

embeddings = []
for tokens in merged_df['tokens']:
    sentence_embeddings = []
    for word in tokens:
        if word in model1.wv:
            sentence_embeddings.append(model1.wv[word])

    # Pad or truncate the sentence embeddings to the target shape
    while len(sentence_embeddings) < target_shape[0]:
        sentence_embeddings.append(np.zeros(target_shape[1]))
    sentence_embeddings = sentence_embeddings[:target_shape[0]]

    embeddings.append(np.array(sentence_embeddings))

# Convert the embeddings to a NumPy array
x = np.array(embeddings)

In [ ]:
embeddings = []
for tokens in merged_df['tokens']:
    sentence_embeddings = []
    for word in tokens:
        if word in model1.wv:
            sentence_embeddings.append(model1.wv[word])
    # Pad or truncate the sentence embeddings to a fixed size of 20 x 300
    if len(sentence_embeddings) < 20:
        padding = np.zeros((20 - len(sentence_embeddings), 100))
        sentence_embeddings.extend(padding)
    else:
        sentence_embeddings = sentence_embeddings[:20]
    embeddings.append(np.array(sentence_embeddings))

# Convert the embeddings to a NumPy array
# x = np.array(embeddings)

<ipython-input-73-f3d790b369b0>:13: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  embeddings.append(np.array(sentence_embeddings))


In [ ]:
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Concatenate, Dense, Dropout

In [ ]:
# Define the model
model = Sequential()

# Add the first convolutional layer
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(20,200)))

# Add the second convolutional layer
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))

# Add the third convolutional layer
model.add(Conv1D(filters=256, kernel_size=3, activation='relu'))

# Add global max pooling layer
model.add(GlobalMaxPooling1D())

# Add a dense layer
model.add(Dense(units=128, activation='relu'))

# Add the output layer
model.add(Dense(units=2, activation='softmax'))

# Print the model summary
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_3 (Conv1D)           (None, 18, 64)            38464     
                                                                 
 conv1d_4 (Conv1D)           (None, 16, 128)           24704     
                                                                 
 conv1d_5 (Conv1D)           (None, 14, 256)           98560     
                                                                 
 global_max_pooling1d_1 (Glo  (None, 256)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 2)                 258       
                                                      

In [ ]:
y = merged_df['Indicator'].values

In [ ]:
y = [[1, 0] if value == 1 else [0, 1] for value in y]

In [ ]:
y=np.array(y)
x=np.array(x)

In [ ]:
# y = np.eye(2)[y]

In [ ]:
print(x.shape)
print(y.shape)

(13641, 20, 200)
(13641, 2)


In [ ]:
x = np.asarray(x)

In [ ]:
print(x.shape)

(13641, 20, 200)


In [ ]:
import tensorflow as tf
y=tf.convert_to_tensor(y)
x=tf.convert_to_tensor(x)

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(x, y, batch_size=256, epochs=25)

Epoch 1/25
54/54 [==============================] - 4s 61ms/step - loss: 0.6946 - accuracy: 0.5077
Epoch 2/25
54/54 [==============================] - 4s 73ms/step - loss: 0.6925 - accuracy: 0.5162
Epoch 3/25
54/54 [==============================] - 3s 61ms/step - loss: 0.6918 - accuracy: 0.5198
Epoch 4/25
54/54 [==============================] - 3s 60ms/step - loss: 0.6916 - accuracy: 0.5209
Epoch 5/25
54/54 [==============================] - 4s 69ms/step - loss: 0.6886 - accuracy: 0.5332
Epoch 6/25
54/54 [==============================] - 4s 64ms/step - loss: 0.6872 - accuracy: 0.5472
Epoch 7/25
54/54 [==============================] - 3s 61ms/step - loss: 0.6801 - accuracy: 0.5636
Epoch 8/25
54/54 [==============================] - 3s 60ms/step - loss: 0.6738 - accuracy: 0.5781
Epoch 9/25
54/54 [==============================] - 4s 73ms/step - loss: 0.6577 - accuracy: 0.6046
Epoch 10/25
54/54 [==============================] - 3s 61ms/step - loss: 0.6437 - accuracy: 0.6230
Epoch 11/

In [ ]:
embeddings = []
for tokens in news_test['tokens']:
    sentence_embeddings = []
    for word in tokens:
        if word in model1.wv:
            sentence_embeddings.append(model1.wv[word])
    # Pad or truncate the sentence embeddings to a fixed size of 20 x 300
    if len(sentence_embeddings) < 20:
        padding = np.zeros((20 - len(sentence_embeddings), 100))
        sentence_embeddings.extend(padding)
    else:
        sentence_embeddings = sentence_embeddings[:20]
    embeddings.append(sentence_embeddings)

# Convert the embeddings to a NumPy array
x_test = np.array(embeddings)

In [ ]:
y_test = news_test['sentiment'].values
y_test = [[1, 0] if value == 1 else [0, 1] for value in y_test]
y_test=np.array(y_test)

In [ ]:
y_pred=model.predict(x)

427/427 [==============================] - 2s 4ms/step


In [ ]:
df.loc[df['Column1'] > df['Column2'], 'Column1'] = 1
df.loc[df['Column1'] <= df['Column2'], 'Column1'] = 0

In [ ]:
print(y_pred.shape)

(13641, 2)


In [ ]:
condition = tf.greater(y_pred[:, 0], y_pred[:, 1])

# Create tensors for [1.0, 0.0] and [0.0, 1.0]
ones_tensor = tf.constant([1.0, 0.0], dtype=tf.float32)
zeros_tensor = tf.constant([0.0, 1.0], dtype=tf.float32)

# Use tf.where to update 'your_tensor' based on the condition
updated_tensor = tf.where(condition[:, tf.newaxis], ones_tensor, zeros_tensor)

# Print the updated tensor
print(updated_tensor)

tf.Tensor(
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [1. 0.]
 [1. 0.]], shape=(13641, 2), dtype=float32)


In [ ]:
tensor_df = pd.DataFrame(updated_tensor.numpy(), columns=['Mpos', 'Mneg'])

In [ ]:
merged_df

,Indicator,Title,tokens
2017-01-02,1,Budget 2017 : Balancing growth with fiscal pru...,"[Budget, 2017, :, Balancing, growth, with, fis..."
2017-01-02,1,Union Budget 2017,"[Union, Budget, 2017]"
2017-01-02,1,Technical Analysis Premium Mailer : Premium St...,"[Technical, Analysis, Premium, Mailer, :, Prem..."
2017-01-02,1,ICICI Bank resumes UPI transactions on PhonePe,"[ICICI, Bank, resumes, UPI, transactions, on, ..."
2017-01-02,1,Mcap of BSE,"[Mcap, of, BSE]"
...,...,...,...
2023-09-01,-1,"Nifty : Nifty may trade in 17 , 777 - 18 , 250...","[Nifty, :, Nifty, may, trade, in, 17, ,, 777, ..."
2023-09-01,-1,Best tax saving options : Comparison of 10 inv...,"[Best, tax, saving, options, :, Comparison, of..."
2023-09-04,1,Trade setup for Monday : Top 15 things to know...,"[Trade, setup, for, Monday, :, Top, 15, things..."
2023-09-04,1,Lenders likely to post robust Q4 numbers ; PSU...,"[Lenders, likely, to, post, robust, Q4, number..."


In [ ]:
tensor_df.index=merged_df.index

In [ ]:
# .reset_index(drop=True, inplace=True)
merged_df.reset_index(drop=True)

,Indicator,Title,tokens
0,1,Budget 2017 : Balancing growth with fiscal pru...,"[Budget, 2017, :, Balancing, growth, with, fis..."
1,1,Union Budget 2017,"[Union, Budget, 2017]"
2,1,Technical Analysis Premium Mailer : Premium St...,"[Technical, Analysis, Premium, Mailer, :, Prem..."
3,1,ICICI Bank resumes UPI transactions on PhonePe,"[ICICI, Bank, resumes, UPI, transactions, on, ..."
4,1,Mcap of BSE,"[Mcap, of, BSE]"
...,...,...,...
13636,-1,"Nifty : Nifty may trade in 17 , 777 - 18 , 250...","[Nifty, :, Nifty, may, trade, in, 17, ,, 777, ..."
13637,-1,Best tax saving options : Comparison of 10 inv...,"[Best, tax, saving, options, :, Comparison, of..."
13638,1,Trade setup for Monday : Top 15 things to know...,"[Trade, setup, for, Monday, :, Top, 15, things..."
13639,1,Lenders likely to post robust Q4 numbers ; PSU...,"[Lenders, likely, to, post, robust, Q4, number..."


In [ ]:
result_df = pd.concat([merged_df, tensor_df], axis=1)

In [ ]:
print(result_df[10:35])

            Mpos  Mneg  Formula_Result_x  Formula_Result_y
2017-01-02   1.0   0.0               1.0          1.000000
2017-01-02   1.0   0.0               1.0          1.000000
2017-01-02   1.0   0.0               1.0          1.000000
2017-01-02   1.0   0.0               1.0          1.000000
2017-01-02   1.0   0.0               1.0          1.000000
2017-01-02   1.0   0.0               1.0          1.000000
2017-01-02   1.0   0.0               1.0          1.000000
2017-01-02   1.0   0.0               1.0          1.000000
2017-01-03   0.0   1.0              -1.0         -0.846154
2017-01-03   0.0   1.0              -1.0         -0.846154
2017-01-03   1.0   0.0               1.0         -0.846154
2017-01-03   0.0   1.0              -1.0         -0.846154
2017-01-03   0.0   1.0              -1.0         -0.846154
2017-01-03   0.0   1.0              -1.0         -0.846154
2017-01-03   0.0   1.0              -1.0         -0.846154
2017-01-03   0.0   1.0              -1.0         -0.8461

In [ ]:
result_df=result_df.drop(['Indicator','Title','tokens'],axis=1)

In [ ]:
formula_results = result_df.groupby(result_df.index).apply(lambda group: (group['Mpos'].sum() - group['Mneg'].sum()) / (group['Mpos'].sum() + group['Mneg'].sum()))

# Create a new DataFrame with unique dates and their corresponding formula results
unique_dates = result_df.index.unique()
new_df = pd.DataFrame({'Date': unique_dates, 'Formula_Result': formula_results})

In [ ]:
print(new_df)

                  Date  Formula_Result
2017-01-02  2017-01-02        1.000000
2017-01-03  2017-01-03       -0.846154
2017-01-04  2017-01-04        1.000000
2017-01-05  2017-01-05       -1.000000
2017-01-06  2017-01-06        0.888889
...                ...             ...
2023-08-02  2023-08-02       -1.000000
2023-08-03  2023-08-03        0.600000
2023-08-04  2023-08-04       -1.000000
2023-09-01  2023-09-01       -0.800000
2023-09-04  2023-09-04        0.333333

[1243 rows x 2 columns]


In [ ]:
new_df.to_csv('ICICI_Sentiments.csv', index=False)

In [ ]:
y_res = np.where(y_pred[:, 0] > y_pred[:, 1], 1, -1)

In [ ]:
df = pd.DataFrame(y_res, columns=['sentiment'])


In [ ]:
news_test.insert(loc=1, column='sentiments', value=df)

In [ ]:
print(news_test.head())

             Date  sentiments  \
190575 2021-05-01         NaN   
190576 2021-05-01         NaN   
190577 2021-05-01         NaN   
190578 2021-05-01         NaN   
190579 2021-05-01         NaN   

                                                    Title  sentiment  \
190575  recovery : Economy likely to maintain V - shap...          1   
190576  Assam government plans to extend two - wheeler...         -1   
190577  Prime Minister Narendra Modi set to brainstorm...          1   
190578  Joe Biden : Asia Society urges Joe Biden to ex...          1   
190579  Haryana government set to provide one percent ...         -1   

                                                   tokens  
190575  [recovery, :, Economy, likely, to, maintain, V...  
190576  [Assam, government, plans, to, extend, two, -,...  
190577  [Prime, Minister, Narendra, Modi, set, to, bra...  
190578  [Joe, Biden, :, Asia, Society, urges, Joe, Bid...  
190579  [Haryana, government, set, to, provide, one, p...  


In [ ]:
news=news.drop(['Title','tokens'],axis=1)

In [ ]:
print(news)

             Date  sentiment
0      2017-05-01          1
1      2017-05-01         -1
2      2017-05-01          1
3      2017-05-01         -1
4      2017-05-01          1
...           ...        ...
200495 2021-12-04          1
200496 2021-11-04         -1
200497 2021-11-04         -1
200498 2021-11-04          1
200499 2021-11-04         -1

[200500 rows x 2 columns]


In [ ]:
agg_df = news.groupby('Date').agg(pos=('sentiment', lambda x: sum(x == 1)),
                                neg=('sentiment', lambda x: sum(x == -1)))

In [ ]:
agg_df['ratio'] = (agg_df['pos'] - agg_df['neg']) / (agg_df['pos'] + agg_df['neg'])


In [ ]:
print(agg_df)

               ratio
Date                
2017-01-01 -0.166667
2017-01-02  0.067227
2017-01-03 -0.091787
2017-01-04 -0.142857
2017-01-05 -0.034483
...              ...
2021-11-04 -0.333333
2021-12-01 -0.153846
2021-12-02 -0.150000
2021-12-03 -0.245902
2021-12-04 -0.015385

[1554 rows x 1 columns]


In [ ]:
agg_df=agg_df.drop(['pos','neg'],axis=1)

In [ ]:
csv_filename = 'output.csv'
agg_df.to_csv(csv_filename)